<a href="https://colab.research.google.com/github/booluckgmie/training/blob/main/Reverse_geocodetoDUN_Parliment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reverse Geocoder Coordinate to District/DUN/Parliment using KAWASANKU API

## Install Library

In [1]:
import pandas as pd
from tqdm import tqdm
import urllib.request
import json
from shapely.geometry import shape, Point
import time

time_start = time.time()
tqdm.pandas()

In [2]:
PATH_GEOJSON = 'https://raw.githubusercontent.com/dosm-malaysia/data-open/main/datasets/geodata/'

geojsons = ['administrative_0_malaysia',
            'administrative_1_state',
            'administrative_2_district',
            'electoral_0_parlimen',
            'electoral_1_dun']

for i in range (len(geojsons)): geojsons[i] = PATH_GEOJSON + geojsons[i] + '.geojson'

states = json.load(urllib.request.urlopen(geojsons[1]))
districts = json.load(urllib.request.urlopen(geojsons[2]))
parlimens = json.load(urllib.request.urlopen(geojsons[3]))
duns = json.load(urllib.request.urlopen(geojsons[4]))
int_jsonfile = {1: states, 2: districts, 3: parlimens, 4: duns}

def reverse_geocode(lon,lat,geojson_file,name_field):
    try: point = Point(lon, lat)
    except Exception as e:
        print(e)
        return 'Error'

    for feature in geojson_file['features']:
        polygon = shape(feature['geometry'])
        if polygon.contains(point): return (feature['properties'])[name_field].title()

    return 'OUT_OF_BOUNDS'

## Bulk Search

In [3]:
#sample data
data = [
['Pulai Chondong',102.2465076,5.808971712],
['Pendang',100.4740906,5.98657645],
['Taiping',100.7365609,4.849121702],
['Padang Tengku',101.9811058,4.230687014],
['Kinabatangan',117.861843,5.587961659],
]

In [4]:
data = pd.DataFrame(data,columns=['name','lon','lat'])
data.head()

,name,lon,lat
0,Pulai Chondong,102.246508,5.808972
1,Pendang,100.474091,5.986576
2,Taiping,100.736561,4.849122
3,Padang Tengku,101.981106,4.230687
4,Kinabatangan,117.861843,5.587962


In [5]:
data.to_csv("sample.csv")

In [6]:
df = data
df = pd.read_csv("sample.csv")

In [7]:
levels = ['country','state','district','parlimen','dun']
for i in [1,2,3,4]: 
    df[levels[i]] = df.progress_apply(lambda x: reverse_geocode(x['lon'], x['lat'], int_jsonfile[i], levels[i]), axis = 1)
df.head()

100%|██████████| 5/5 [00:00<00:00, 58.14it/s]


,name,lon,lat,state,district,parlimen,dun
0,Pulai Chondong,102.246508,5.808972,Kelantan,Machang,P.029 Machang,N.33 Pulai Chondong
1,Pendang,100.474091,5.986576,Kedah,Pendang,P.011 Pendang,N.18 Tokai
2,Taiping,100.736561,4.849122,Perak,Larut Dan Matang,P.060 Taiping,N.17 Pokok Assam
3,Padang Tengku,101.981106,4.230687,Pahang,Lipis,P.079 Lipis,N.03 Padang Tengku
4,Kinabatangan,117.861843,5.587962,Sabah,Kinabatangan,P.187 Kinabatangan,N.58 Lamag


In [8]:
df.to_csv('sampleMapping.csv', index=False)

## Single Search

In [13]:
print("insert your geocode")
insert = input()

insert your geocode
5.808972,102.246508


In [14]:
data = insert
df = pd.DataFrame([x.split(',') for x in data.split('\n')])
df.columns =['lat', 'long']
print(df)

        lat        long
0  5.808972  102.246508


In [15]:
data = [
[df['long'],df['lat']]
]

df = pd.DataFrame(data,columns=['long','lat'])
df.head()

,long,lat
0,"0 102.246508 Name: long, dtype: object","0 5.808972 Name: lat, dtype: object"


In [16]:
levels = ['country','state','district','parlimen','dun']
for i in [1,2,3,4]: 
    df[levels[i]] = df.progress_apply(lambda x: reverse_geocode(x['long'], x['lat'], int_jsonfile[i], levels[i]), axis = 1)
df.head()

100%|██████████| 1/1 [00:00<00:00, 86.38it/s]


,long,lat,state,district,parlimen,dun
0,"0 102.246508 Name: long, dtype: object","0 5.808972 Name: lat, dtype: object",Kelantan,Machang,P.029 Machang,N.33 Pulai Chondong
